In [13]:
import torch
import numpy as np
import os
import py3Dmol
from Bio.PDB import Structure, Model as PDBModel, Chain, Residue, Atom
from Bio.PDB.PDBIO import PDBIO
from torch_geometric.data import Data

from dataset import DNADataset
from model import Model
from config import DEVICE

In [8]:
dataset = DNADataset()

# Инициализируем модель с той же архитектурой, что и при обучении
# +3 для координат (x, y, z)
node_dim = dataset.num_node_features + 3
model = Model(node_dim=node_dim, edge_dim=1, hidden_dim=64, num_timesteps=200)

# Загружаем обученные веса
# Убедитесь, что файл dna_diffusion_model.pth находится в той же папке
try:
    model.load_state_dict(torch.load(os.path.join('..', 'model.pth'), map_location=DEVICE))
except FileNotFoundError:
    print("Ошибка: Файл 'model.pth' не найден. Убедитесь, что вы запустили train.py и он успешно завершился.")

model = model.to(DEVICE)
model.eval()

print("Модель и датасет успешно загружены.")

API request resulted in 2599 PDB IDs


  5%|▌         | 5/100 [00:08<02:34,  1.62s/it]


KeyboardInterrupt: 

In [ ]:
# Возьмем первый элемент датасета для примера
original_full_graph = dataset[0].to(DEVICE)

# 1. Создаем граф-условие (азотистые основания и фланкирующие остовы)
is_condition = ~original_full_graph.central_mask
condition_graph = original_full_graph.subgraph(is_condition)

# 2. Определяем, сколько узлов нужно сгенерировать (атомы центрального остова)
is_target = original_full_graph.backbone_mask & original_full_graph.central_mask
num_nodes_to_generate = is_target.sum().item()

# 3. Запускаем сэмплинг
print(f"Генерация структуры с {num_nodes_to_generate} атомами...")
with torch.no_grad():
    generated_nodes, generated_pos, generated_edges = model.sample(
        condition_graph, num_nodes=num_nodes_to_generate
    )

print(f"Сгенерирована структура с {generated_nodes.shape[0]} атомами, {generated_pos.shape[0]} позициями и {generated_edges.shape[1]} ребрами.")

In [ ]:
# Словарь для сопоставления индекса и типа атома
idx_to_atom = {v: k for k, v in dataset.atom_to_idx.items()}


def graph_to_cif(atom_types, pos, file_path):
    """Конвертирует данные графа в файл .cif с помощью BioPython."""
    struct = Structure.Structure('generated_structure')
    model = PDBModel.Model(0)
    chain = Chain.Chain('A')

    # Просто помещаем все атомы в один остаток для простоты
    residue = Residue.Residue((' ', 1, ' '), 'UNK', ' ')

    for i, (atom_type_idx, coord) in enumerate(zip(atom_types, pos)):
        atom_name = idx_to_atom.get(atom_type_idx, 'X')  # 'X' для неизвестных
        # BioPython требует, чтобы имя атома было уникальным, добавим индекс
        unique_atom_name = f"{atom_name}{i+1}"
        atom = Atom.Atom(
            name=unique_atom_name,
            coord=coord,
            bfactor=0,
            occupancy=1.0,
            altloc=' ',
            fullname=unique_atom_name,  # Используем уникальное имя
            ser_num=i+1,
            element=atom_name.strip()
        )
        residue.add(atom)

    chain.add(residue)
    model.add(chain)
    struct.add(model)

    io = PDBIO()
    io.set_structure(struct)
    io.save(file_path, format='cif')
    print(f"Структура сохранена в {file_path}")


# Конвертируем сгенерированную структуру
gen_atom_types = torch.argmax(generated_nodes, dim=1).cpu().numpy()
gen_pos = generated_pos.cpu().numpy()
graph_to_cif(gen_atom_types, gen_pos, 'generated.cif')

# Конвертируем оригинальную целевую структуру
original_target_graph = original_full_graph.subgraph(is_target)
orig_atom_types = torch.argmax(original_target_graph.x, dim=1).cpu().numpy()
orig_pos = original_target_graph.pos.cpu().numpy()
graph_to_cif(orig_atom_types, orig_pos, 'original.cif')